<a href="https://colab.research.google.com/github/JeroAgullo/fundamental_valuation/blob/main/fundamental_valuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Fundamental Valuation using Python

## 1.1. Introduction: let’s torch the hype

**Project Approach**

Welcome to yet another boring finance project - except this one is spiced with Python and data-driven bitterness. We're going to dissect NVIDIA’s (NVDA) financial guts using the Yahoo Finance API via the **yfinance** library, because scraping data like it is 2010 is a hobby of people who enjoy pain.

The notebook parses and standardizes key financial statement elements into a format that facilitates **ratio analysis and valuation modeling**, which is specially useful for dummies like me.

**Fundamental analysis**

Yes, the ancient art of pretending we can outsmart the market by estimating the **intrinsic value of a security**. Because hey, if Buffett does it, it must work, right?

Unlike technical analysis, which is astrology for traders, fundamental analysis pretends to find the “real” value of a stock and **compare it with its current price**. Spoiler: the market doesn’t care anymore about our spreadsheets.

## 1.2. First steps: stalking the company

Let’s import some libraries like responsible nerds and pretend we’re doing something no one else has thought of before.

In [5]:
import yfinance as yf
import pandas as pd

Now, let’s summon the almighty NVDA ticker (the market "ID") like some Wall Street necromancer.

Just in case any fanboy would like to run this code for their favourite capitalist society, here is a list of the current companies in the S&P500 Index, and their ticker:

https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

In [6]:
ticker = yf.Ticker("NVDA")

Time to check what is inside the monster. This isn't magic; it is just unpaid interns at Yahoo structuring your API access.

Spoiler alert: You will get a tsunami of financial jargon you will pretend to understand to impress LinkedIn recruiters. Comment this code if you get bored of scrolling.

In [7]:
headers = ticker.balance_sheet.index.tolist() # We will print the headers of the Income Statement report
for h in headers:
  print(h)

Treasury Shares Number
Ordinary Shares Number
Share Issued
Net Debt
Total Debt
Tangible Book Value
Invested Capital
Working Capital
Net Tangible Assets
Capital Lease Obligations
Common Stock Equity
Total Capitalization
Total Equity Gross Minority Interest
Stockholders Equity
Gains Losses Not Affecting Retained Earnings
Other Equity Adjustments
Treasury Stock
Retained Earnings
Additional Paid In Capital
Capital Stock
Common Stock
Preferred Stock
Total Liabilities Net Minority Interest
Total Non Current Liabilities Net Minority Interest
Other Non Current Liabilities
Employee Benefits
Tradeand Other Payables Non Current
Non Current Deferred Liabilities
Non Current Deferred Revenue
Non Current Deferred Taxes Liabilities
Long Term Debt And Capital Lease Obligation
Long Term Capital Lease Obligation
Long Term Debt
Current Liabilities
Other Current Liabilities
Current Deferred Liabilities
Current Deferred Revenue
Current Debt And Capital Lease Obligation
Current Capital Lease Obligation
Curre

## 1.3. Data extraction: AKA copy-pasting numbers from the Internet

Welcome to the exciting world of extracting too much data, only to use five values in your report for your boss. We will select specific data from each Financial State (Financials):

- Income Statement
- Balance Sheet
- Cashflow

The market information (such as price or number of shares) will be also needed. Probably you were looking just for this, right?

First, a little function that avoids blowing up your notebook when Yahoo decides not to play nice:

In [8]:
def safe_extract(df, row_name, column_name):
    if row_name in df.index:
        return df.loc[row_name, column_name]
    else:
        return 0

Data extract (1/4) - Income Statement: are they even making any money?

In [9]:
df = ticker.ttm_financials
column_name = df.columns[0] # First column from the TTM (Trailing Twelve Months) report

total_revenue = safe_extract(df, "Total Revenue", column_name)
gross_profit = safe_extract(df, "Gross Profit", column_name)
ebitda = safe_extract(df, "EBITDA", column_name)
ebit = safe_extract(df, "EBIT", column_name)
net_income = safe_extract(df, "Net Income", column_name)

Data extract (2/4) - Balance Sheet: how rich are they, really?

In [10]:
df= ticker.balancesheet
column_name = df.columns[0] # First column, to get only the latest accounting period report

total_assets = safe_extract(df, "Total Assets", column_name)
current_assets = safe_extract(df, "Current Assets", column_name)
inventory = safe_extract(df, "Inventory", column_name)
total_liabilities = safe_extract(df, "Total Liabilities Net Minority Interest", column_name)
current_liabilities = safe_extract(df, "Current Liabilities", column_name)
total_equity = safe_extract(df, "Total Equity Gross Minority Interest", column_name)
cash = safe_extract(df, "Cash And Cash Equivalents", column_name)
ordinary_shares = safe_extract(df, "Ordinary Shares Number", column_name)

Data extract (3/4) - Cashflow: because even cool companies need money

In [11]:
df = ticker.ttm_cash_flow
column_name = df.columns[0] # First column from the TTM (Trailing Twelve Months) report

cashflow = safe_extract(df, "Operating Cash Flow", column_name)
free_cashflow = safe_extract(df, "Free Cash Flow", column_name)

Data extract (4/4) - Market information: the delusional price people pay

In [12]:
df = ticker.info

current_price = df["currentPrice"]
current_shares = df["sharesOutstanding"]
current_market_cap = df["marketCap"]

Now let's flex these numbers like we're pitching in a business podcast:

In [13]:
print("Income Statement results")
print(f"Total Revenue: {total_revenue:,.0f}$")
print(f"Total Profit: {gross_profit:,.0f}")
print(f"EBITDA: {ebitda:,.0f}$")
print(f"EBIT: {ebit:,.0f}$")
print(f"Net Income: {net_income:,.0f}$")

print("\nBalance Sheet results")
print(f"Total Assets: {total_assets:,.0f}$")
print(f"Current Assets: {current_assets:,.0f}$")
print(f"Inventory: {inventory:,.0f}$")
print(f"Total Liabilities: {total_liabilities:,.0f}$")
print(f"Current Liabilities: {current_liabilities:,.0f}$")
print(f"Total Equity: {total_equity:,.0f}$")
print(f"Cash: {cash:,.0f}$")


print("\nCashflow results")
print(f"Cash Flow: {cashflow:,.0f}$")
print(f"Free Cash Flow: {free_cashflow:,.0f}$")

print("\nMarket information")
print(f"Current Price: {current_price:,.2f}$")
print(f"Current Shares: {current_shares:,.0f}")
print(f"Current Market Cap: {current_market_cap:,.0f}$")

Income Statement results
Total Revenue: 148,515,000,000$
Total Profit: 104,120,000,000
EBITDA: 90,968,000,000$
EBIT: 88,903,000,000$
Net Income: 76,774,000,000$

Balance Sheet results
Total Assets: 111,601,000,000$
Current Assets: 80,126,000,000$
Inventory: 10,080,000,000$
Total Liabilities: 32,274,000,000$
Current Liabilities: 18,047,000,000$
Total Equity: 79,327,000,000$
Cash: 8,589,000,000$

Cashflow results
Cash Flow: 76,158,000,000$
Free Cash Flow: 72,064,000,000$

Market information
Current Price: 173.72$
Current Shares: 24,387,600,384
Current Market Cap: 4,236,614,041,600$


If you're still reading, congrats. You've officially spent more time analyzing NVIDIA than half the meme-stock crowd.

# 2. Financial Ratios

## 2.1. Introduction: pretending numbers reveal truth

Financial ratios are quantitative metrics derived from Financial Statements' data, allowing ChatGPT users like me to simplify the interpretation of such raw information.

These ratios are a key part in this fundamental valuation thing, and they can be structured as followed:

- Leverage and Liquidity Ratios
- Profitability Ratios
- Cash Flow
- Valuation Ratios (Multiples)

## 2.2. Leverage and Liquidity Ratios: can they survive a bad quarter?

Indicators that let us know the company's the strength of its capital structure and the capacity to attend to its long-term (leverage) and its short-term obligations (liquidity).

Basically, they let us know if the company can pay their bills without selling the furniture.

In [14]:
# Leverage Ratios
debt_equity = total_liabilities / total_equity
print(f"Debt to Equity (D/E) Ratio: {debt_equity:.2%}")

debt_asset = total_liabilities / total_assets
print(f"Debt to Asset (D/A) Ratio: {debt_asset:.2%}")

Debt to Equity (D/E) Ratio: 40.68%
Debt to Asset (D/A) Ratio: 28.92%


In [15]:
# Liquidity Ratios
current = current_assets / current_liabilities
print(f"Current Ratio: {current:.2%}")

quick = (current_assets - inventory) / current_liabilities
print(f"Quick (Acid) Ratio: {quick:.2%}")

cash = cash / current_liabilities
print(f"Cash Ratio: {cash:.2%}")

Current Ratio: 443.99%
Quick (Acid) Ratio: 388.13%
Cash Ratio: 47.59%


## 2.3. Profitability Ratios: making money isn't enough anymore

Indicators that let us know, not only if the company can attend to its obligations, but also if it gives profit to its owners.

Are they actually doing something with their asset or just hoarding them? Is there any real profit before the taxman shows up? Here are the answers.

In [16]:
# Return Ratios
roi = net_income / total_revenue
print(f"Return on Investment (ROI): {roi:.2%}")

roe = net_income / total_equity
print(f"Return on Equity (ROE): {roe:.2%}")

roa = net_income / total_assets
print(f"Return on Assets (ROA): {roa:.2%}")

Return on Investment (ROI): 51.69%
Return on Equity (ROE): 96.78%
Return on Assets (ROA): 68.79%


In [17]:
# Efficency Ratios
gross_margin = gross_profit / total_revenue
print(f"Gross Margin: {gross_margin:.2%}")

ebitda_margin = ebitda / total_revenue
print(f"EBITDA Margin: {ebitda_margin:.2%}")

fcf_revenue = free_cashflow / total_revenue
print(f"Free Cash Flow Margin: {fcf_revenue:.2%}")

Gross Margin: 70.11%
EBITDA Margin: 61.25%
Free Cash Flow Margin: 48.52%


In [18]:
# Share Ratios
bvps = total_equity / ordinary_shares # We use the share number reported in the same period as the Equity (Accounting Period)
print(f"Book Value per Share (BVPS): {bvps:.2f}$")

eps = net_income / current_shares # We use the share number reported in the same period as the Net Income (TTM)
print(f"Earnings per Share (EPS): {eps:.2f}$")

cfps = cashflow / current_shares # We use the share number reported in the same period as the Cash Flow (TTM)
print(f"Cash Flow per Share (CFPS): {cfps:.2f}$")

Book Value per Share (BVPS): 3.24$
Earnings per Share (EPS): 3.15$
Cash Flow per Share (CFPS): 3.12$


## 2.4 Cash Flow: the only metric that should matter

The cash flow reflects a company's ability to generate liquidity from its operations, financing, and investing activities.

This is what investors would care about if they weren’t too busy chasing AI stonks and meme coins.

In [19]:
print(f"Cash Flow: {cashflow:,.0f}$")

print(f"Free Cash Flow: {free_cashflow:,.0f}$")

fcf_yield = free_cashflow / current_market_cap
print(f"Free Cash Flow Yield: {fcf_yield:.2%}")

Cash Flow: 76,158,000,000$
Free Cash Flow: 72,064,000,000$
Free Cash Flow Yield: 1.70%


## 2.5. Valuation Ratios (Multiples): now entering Wonderland

Indicators most commonly used in financial investments. They allow us to compare companies from the same sector and competitors in the market.

They just sound nice and smart, stock price will do whatever it wants anyway.

In [20]:
# Price to Book Value (PBV)
p_bv = current_price / bvps
print(f"Price to Book Value (P/BV): x{p_bv:.2f}")

Price to Book Value (P/BV): x53.60


In [21]:
# Price to Earnings Ratio (PER)
per = current_price / eps
print(f"Price to Earnings Ratio (PER or P/E): x{per:.2f}")

Price to Earnings Ratio (PER or P/E): x55.18


In [22]:
# Enterprise Value to EBITDA (EV/EBITDA)
enterprise_value = current_market_cap + total_liabilities - cash # Enterprise Value (EV) must be calculated first
ev_ebitda = enterprise_value / ebitda
print(f"Enterprise Value to EBITDA (EV/EBITDA): x{ev_ebitda:.2f}")

Enterprise Value to EBITDA (EV/EBITDA): x46.93


In [23]:
# Price to Cash Flow per share (PCF)
pcf = current_price / cfps
print(f"Price to Cash Flow (PCF): x{pcf:.2f}")

Price to Cash Flow (PCF): x55.63


# Outro
That’s it, you made it to the end. Congrats, you survived corporate finance!

Questions? Complaints? Passive-aggressive praise? Here is how to blackmail me: jeronimo.civil@gmail.com

In case you somehow found this useful, here is my github user. Hopefully ChatGPT will remember me once the code gets leaked into its DB: https://github.com/JeronimoAgullo

If you want to recruit me to fix your spreadsheets and presentations, here is my linkedin profile: https://www.linkedin.com/in/meetme-jeronimoagullo